In [1]:
#Merging all restaurant data 

import glob
import pandas as pd

# get data file names
path =r"C:/Users/YESIM/Desktop/TEXT MINING PROJECT/AMSTERDAM/RESTAURANTS/All"
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, error_bad_lines=False) )

# Concatenate all data into one DataFrame
df = pd.concat(dfs, ignore_index=True)



b'Skipping line 4: expected 5 fields, saw 6\nSkipping line 5: expected 5 fields, saw 6\nSkipping line 6: expected 5 fields, saw 6\nSkipping line 11: expected 5 fields, saw 6\nSkipping line 12: expected 5 fields, saw 6\nSkipping line 26: expected 5 fields, saw 6\nSkipping line 29: expected 5 fields, saw 6\nSkipping line 37: expected 5 fields, saw 6\nSkipping line 46: expected 5 fields, saw 6\nSkipping line 55: expected 5 fields, saw 6\nSkipping line 66: expected 5 fields, saw 6\nSkipping line 68: expected 5 fields, saw 6\nSkipping line 77: expected 5 fields, saw 6\nSkipping line 88: expected 5 fields, saw 6\nSkipping line 96: expected 5 fields, saw 6\nSkipping line 113: expected 5 fields, saw 6\nSkipping line 114: expected 5 fields, saw 6\nSkipping line 115: expected 5 fields, saw 6\nSkipping line 116: expected 5 fields, saw 6\nSkipping line 121: expected 5 fields, saw 6\nSkipping line 124: expected 5 fields, saw 6\nSkipping line 143: expected 5 fields, saw 6\nSkipping line 153: expecte

In [2]:
#Data Preparation

import numpy as np 

df = df.fillna(value=str(0))

df["full_text"] = df["title"] + ". " + df["review"]

#Reviews with 3 stars or lower are negative sentiment and others are positive sentiment 
def get_class(stars):
    score = int(stars[0])
    if score > 3:
        return 1
    else:
        return 0

def review_count(review_count):
    return review_count[0]

def help_count(help_count):
    return help_count[0]

def del_more(full_text):
    if full_text[-1] == "e" and full_text[-2] == "r" and full_text[-3] == "o" and full_text[-4] == "M":
        return full_text[0:-8]
    else:
        return full_text
    
df["review count"] = df["review_count"].apply(review_count).astype(int)
df["help count"] = df["help_count"].apply(help_count).astype(int)
df["sentiment"] = df["stars"].apply(get_class)
df["id"] = range(0,len(df))
df["full_text"] = df["full_text"].apply(del_more)

df_sent = df[["full_text", "sentiment"]]

len(df_sent[df_sent["sentiment"]==0])



2100

In [3]:
#PREPROCESS ARGUMENT OF CLASS NONE OR FUNCTION

#Preprocess, Tokenize and Remove Duplicates
import re
from nltk.corpus import stopwords
stop = stopwords.words("english")

def preprocess(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', '')    
    text = re.sub("[',.,!,?,&,%,$,@,(,)]", "", text)
    text = text.lower()
    text = [word for word in text.split() if word not in stop]
    text = list(set(text))
    return text

df_sent["full_text"] = df["full_text"].apply(preprocess)

len(df_sent[df_sent["sentiment"]==0])/len(df_sent[df_sent["sentiment"]==1])



C:\Users\YESIM\Anaconda3\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.17884517118037813

In [4]:
#Split data into train and test

X = df_sent["full_text"].values
y = df_sent["sentiment"].values
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.2 ,random_state=1)

# Keep X_test and y_test until the last step
X_test
y_test

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [5]:
#Cross Validation k = 5

from sklearn.cross_validation import StratifiedKFold

skf = StratifiedKFold(y_train, n_folds = 5, random_state = 42)

train_indexes = []
val_indexes = []
for train_index, val_index in skf:
    train_indexes.append(train_index)
    val_indexes.append(val_index)


a = 5
a

5

In [6]:
# 5 different train and val sets before the actual y_test
#1st fold

X_train_skf = X_train[train_indexes[1]]
y_train_skf = y_train[train_indexes[1]]
X_val = X_train[val_indexes[1]]
y_val = y_train[val_indexes[1]]

a = 5
a

5

In [7]:
#FREQ METHODS OF CLASS -- WHAT TO DO FOR SAMPLES WITH MORE CLASS ?
# Training set consisting individual reviews and classes
X_train_skf
y_train_skf


#Counting distinct words in both classes


X_train_0 = []
X_train_1 = []

for lists in X_train_skf[y_train_skf==0]:
    for words in lists:
        X_train_0.append(words)
for lists in X_train_skf[y_train_skf==1]:
    for words in lists:
        X_train_1.append(words)    
        
from collections import Counter
counts_0 = Counter(X_train_0)
counts_1 = Counter(X_train_1)

#Calculate P(C) for C = 0,1
class0_prob = len(X_train_0)/len(X_train_0+X_train_1)
class1_prob = len(X_train_1)/len(X_train_0+X_train_1)


#Calculate class freq of each word f(W,C)
freq_0_list = []
freq_1_list = []

for key in counts_0:
    array = []
    array.append(key)
    array.append(counts_0[key]/len(X_train_0))
    freq_0_list.append(array)

for key in counts_1:
    array = []
    array.append(key)
    array.append(counts_1[key]/len(X_train_1))
    freq_1_list.append(array)

#Calculate total freq of each words sum(f(W,Ci)) for i=0,1
#FREQ TOTAL I TEKRAR HESAPLAMAK LAZIM

all_counts = Counter(counts_0 + counts_1)

freq_total_list = []

for key in all_counts:
    array = []
    array.append(key)
    array.append( (counts_0[key]/len(X_train_0)) + (counts_1[key]/len(X_train_1)))
    freq_total_list.append(array)             
    
minfreq1 = 0.000000000000001
minfreq0 = 0.000000000000001

a = 5
a

5

In [8]:
#Prediction Method of CLASS

pred = []

for array in X_test:

    freq1 = []
    freq0 = []
    freq_total = []

    for word in array:   

        count = 0
        for lists in freq_1_list:
            if lists[0] == word:
                freq1.append(lists[1])
                break
            else:
                count = count + 1
            if count == len(freq_1_list):
                freq1.append(0)



    for word in array:   

        count = 0
        for lists in freq_0_list:
            if lists[0] == word:
                freq0.append(lists[1])
                break
            else:
                count = count + 1
            if count == len(freq_0_list):
                freq0.append(0)            


    for word in array:   

        count = 0
        for lists in freq_total_list:
            if lists[0] == word:
                freq_total.append(lists[1])
                break
            else:
                count = count + 1
            if count == len(freq_total_list):
                freq_total.append(0)  


    freq_total_1 = list(freq_total)
    freq_total_0 = list(freq_total)

    #Smoothing the frequencies
    for i in range(0,len(freq1)):

        if freq1[i] == 0:
            if freq_total_1[i] == 0:
                freq1[i] = 0.5
                freq_total_1[i] = 1
            else:
                freq1[i] = minfreq1

    for i in range(0,len(freq0)):

        if freq0[i] == 0:
            if freq_total_0[i] == 0:
                freq0[i] = 0.5
                freq_total_0[i] = 1
            else:
                freq0[i] = minfreq1


    score0 = np.sum(np.log10(np.array(freq0)/np.array(freq_total_0))) + np.log10(class0_prob)
    score1 = np.sum(np.log10(np.array(freq1)/np.array(freq_total_1))) + np.log10(class1_prob)

    if score0 > score1:
        pred.append(0)
    else:
        pred.append(1)         

    freq1 = []
    freq0 = []
    freq_total = []
    

a = 5
a

5

In [9]:
#Selecting the best performance among stratified kfolds

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix

cmat = confusion_matrix(y_test, pred)

TN_RATIO = cmat[0][0]/np.sum(cmat[0])




f1_score = f1_score(y_test, pred, average = "micro")
accuracy = accuracy_score(y_test, pred)
precision = precision_score(y_test, pred)
recall = recall_score(y_test, pred)

print("Accuracy: %f\nPrecision: %f\nRecall: %f\nF1 score(micro): %f" %(accuracy, precision ,recall, f1_score))
print("TN/TN+FN: %f" %TN_RATIO )

cmat

Accuracy: 0.898519
Precision: 0.918831
Recall: 0.965458
F1 score(micro): 0.941568
TN/TN+FN: 0.528302


C:\Users\YESIM\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)


array([[ 224,  200],
       [  81, 2264]])